# Agents and Tools

In this section of the lab, we will review some agent concepts and see how agents can interact the basic tools

- Define a basic agent, using the `builtin::websearch` tool from Llama Stack
- Get familiar with concepts like "Turns" and "Steps" and "Tool_Calling" that the agent takes
- Run the agent to return web results
- Optional: experience a prompt chaining example

> **Note:**
>This section is intended only as an introduction to Agent concepts and language, and we will get into more exciting work in the next modules. If you are familiar with these concepts, feel free to skip this section.
> 


> **Note:**
> For the sake of simplicity and time, we will skip through some of the python basics we went through in previous modules and treat this more like a nomral python application 
> 

### Install Python Prerequisist

As always, let's start by installing the Python Libraries we neeed, we will add some additional libraries to support some of our tools. 

In [1]:
!pip install -U llama-stack-client==0.2.5 dotenv geocoder

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [geocoder]1/3 [future]


### Define the LLamastack server and Model

Let's define the libraries we need and our basic Llama Stack configurations:

In [2]:
import os

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# These libraries are just here to print the results from the agent in a more human-readable way 
from src.utils import step_printer
from termcolor import cprint
import uuid
from llama_stack_client.lib.agents.event_logger import EventLogger
stream=False ## Defaulting to False, you can change this throughout the section to "True" if you wanted to see the output in another format (Using EventLogger)

# for our lab, we will just define our variables manualy here, in a regular application, this would be ready directly from the local .env file and we would comment these lines out
os.environ['LLAMA_STACK_SERVER'] = 'http://localhost:8321'

# We will be using the Tavily web search service (docs.tavily.com/)
tavily_search_api_key='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'
provider_data = {"tavily_search_api_key": tavily_search_api_key}

> **Note:**
>When running this code in a regular Python application, we would usually like to read environment variables from an `.env` file, for our needs in this lab, we will hard code these in this cell, to make things more clear
>

### Initialize the *Client* 
We will not define our client, and introduce a new pattern/trick that will help us later when we try to move between models and other providers within Llama Stack. 

In [3]:
LLAMA_STACK_SERVER=os.getenv("LLAMA_STACK_SERVER")

client = LlamaStackClient(
    base_url=LLAMA_STACK_SERVER,
    provider_data=provider_data
)
# List available models
models = client.models.list()
print("--- Available models: ---")
for m in models:
    print(f"{m.identifier} - {m.provider_id} - {m.provider_resource_id}")

# For our development purposes, we might want to change different models and test with them, these lines select the first available model in the Llama Stack server. 
SELECTED_MODEL=models[0].identifier
print("--- Selected model: ---")
print(SELECTED_MODEL)

--- Available models: ---
meta-llama/Llama-3.2-3B-Instruct - ollama - llama3.2:3b-instruct-fp16
all-MiniLM-L6-v2 - ollama - all-minilm:latest
--- Selected model: ---
meta-llama/Llama-3.2-3B-Instruct


### Define the a single tool agent
Here we define our new agent, we will pass along the `client` defining our connection to the Llama Stack server, the `model` we selected in the previous step, the `tools` array with all available tools for the agent, and crucually, the prompt, named `instructions` for the agent.

> **Note:**
> You might notice that we are being very specific in our prompt, for the purpose of lab we are using rather small models and are providing a narrow path to follow, in a real world scenario, prompts and LLMs can be experimented with to produce the best results.
> 

In [4]:
from llama_stack_client.lib.agents.agent import Agent

agent = Agent(
    client, 
    model=SELECTED_MODEL,
    instructions="""You are a helpful websearch assistant. When you are asked to search the latest you must use a tool. 
            Whenever a tool is called, be sure return the response in a friendly and helpful tone.
            """ ,
    tools=["builtin::websearch"],
)

### inspecting the Agent's response structure

Let's give our agent a couple of easy questions to see how it calls our `builtin::websearch` tool.
Notice that we are generating a new session ID for each question as convesation memory is shared throughtout the session. (More on this later!)


In [5]:
user_prompts = [
       "search the web for the latest in OpenShift?",
       "search for the current weather in boston",
]


for prompt in user_prompts:
    # Generate a new Unique Identifier for each session 
    new_uuid = uuid.uuid4()
    session_id = agent.create_session(f"web-session-{new_uuid}")

    cprint(f"\n{'='*100}\nProcessing user query: {prompt}\n{'='*100}", "blue")

    
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: search the web for the latest in OpenShift?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'latest news OpenShift'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'latest news OpenShift',
│   'top_k': [
│   │   {
│   │   │   'title': 'Azure Red Hat OpenShift: April 2025 Update | Microsoft Community Hub',
│   │   │   'url': 'https://techcommunity.microsoft.com/blog/appsonazureblog/azure-red-hat-openshift-april-2025-update/4400331',
│   │   │   'content': "OpenShift 4.16 on Day 0 → Now available for new ARO cluster installs with the latest features and improvements. See what's new. Note that a cluster version upgrade support (Day 2 operation) kicks in for existing clusters as soon as an upgrade path is available. This happens shortly after OCP GA availability for a version.",
│   │   │   'score': 0.769098,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': "What's new for developers in Red Hat OpenShift: Q1 2024 release round-up",
│   │   │   'url': 'https://developers.redhat.com/blog/2024/04/03/whats-new-developers-red-hat-openshift-1st-quarter-2024-release-round',
│   │   │   'content': "Red Hat OpenShift is the application platform to build, modernize, and deploy applications. Red Hat OpenShift 4.15, based on Kubernetes 1.28 and CRI-O 1.28, is now generally available.Review What's new for developers in Red Hat OpenShift 4.15 to get the details on the latest release. To try OpenShift, start with the Developer Sandbox for Red Hat OpenShift or check out our learning content.",
│   │   │   'score': 0.7587435,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Red Hat OpenShift 4.17: What you need to know',
│   │   │   'url': 'https://www.redhat.com/en/blog/what-you-need-to-know-red-hat-openshift-417',
│   │   │   'content': 'Red Hat OpenShift 4.17 is now generally available. Based on Kubernetes 1.30 and CRI-O 1.30, OpenShift 4.17 features expanded control plane options, increased flexibility for virtualization and networking, new capabilities to leverage generative AI, and continued investment in Red Hat OpenShift Platform Plus.These additions further accelerate innovation with OpenShift without compromising on',
│   │   │   'score': 0.6954834,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Red Hat Blends Innovation with Stability in Latest Release of Red Hat ...',
│   │   │   'url': 'https://www.redhat.com/en/about/press-releases/red-hat-blends-innovation-stability-latest-release-red-hat-openshift',
│   │   │   'content': "Red Hat, Inc., the world's leading provider of open source solutions, today introduces new capabilities and enhancements for Red Hat OpenShift, the industry's leading hybrid cloud application platform powered by Kubernetes, as well as the technology preview of Red Hat OpenShift Lightspeed. The new features, delivered with the general",
│   │   │   'score': 0.6708892,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': "What's new in Red Hat OpenShift",
│   │   │   'url': 'https://www.redhat.com/en/whats-new-red-hat-openshift',
│   │   │   'content': 'Red Hat AI A portfolio for developing and deploying artificial intelligence solutions across the hybrid cloud.; Red Hat Enterprise Linux A flexible, stable operating system to support hybrid cloud innovation.; Red Hat OpenShift A container platform to build, modernize, and deploy applications at scale.; Red Hat Ansible Automation Platform A foundation for implementing enterprise-wide automation.',
│   │   │   'score': 0.42935818,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Here's the final answer to the user's question:

The latest news on OpenShift can be found in various sources, including Microsoft Community Hub, Red Hat blog, and Red Hat website. The latest updates include:

* Azure Red Hat OpenShift: April 2025 Update, which is now available for new ARO cluster installs with the latest features and improvements.
* Red Hat OpenShift 4.15, based on Kubernetes 1.28 and CRI-O 1.28, is now generally available.
* Red Hat OpenShift 4.17, based on Kubernetes 1.30 and CRI-O 1.30, features expanded control plane options, increased flexibility for virtualization and networking, new capabilities to leverage generative AI, and continued investment in Red Hat OpenShift Platform Plus.
* Red Hat Blends Innovation with Stability in Latest Release of Red Hat OpenShift.

These updates provide a range of new features and enhancements for developers, including support for artificial intelligence solutions,

{
│   'query': 'current weather in Boston',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in Boston',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Boston', 'region': 'Massachusetts', 'country': 'United States of America', 'lat': 42.3583, 'lon': -71.0603, 'tz_id': 'America/New_York', 'localtime_epoch': 1746734958, 'localtime': '2025-05-08 16:09'}, 'current': {'last_updated_epoch': 1746734400, 'last_updated': '2025-05-08 16:00', 'temp_c': 17.2, 'temp_f': 63.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 7.6, 'wind_kph': 12.2, 'wind_degree': 116, 'wind_dir': 'ESE', 'pressure_mb': 1016.0, 'pressure_in': 30.0, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 72, 'cloud': 75, 'feelslike_c': 17.2, 'feelslike_f': 63.0, 'windchill_c': 16.7, 'windchill_f': 62.1, 'heatindex_c': 16.7, 'heatindex_f': 62.1, 'dewpoint_c': 13.0, 'dewpoint_f': 55.3, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.2, 'gust_mph': 10.5, 'gust_kph': 17.0}}",
│   │   │   'score': 0.9752201,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'May 8 Weather in Boston: Detailed Weather Forecast for May 8 in Boston ...',
│   │   │   'url': 'https://world-weather.info/forecast/usa/boston/08-may/',
│   │   │   'content': 'Detailed ⚡ Weather Forecast for May 8 in Boston, Massachusetts, United States - 🌡️ temperature, wind, atmospheric pressure, humidity and precipitations - World-Weather.info. Add the current city. Search. Weather; Archive; Widgets °F. ... May 06 May 07 Select date: May 09 May 10. May 08, 2025 : Atmospheric conditions and temperature °F:',
│   │   │   'score': 0.84277284,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Weather in Boston - Climate monitor on May 2025',
│   │   │   'url': 'https://www.weatherandclimate.info/monitor/KBOS',
│   │   │   'content': 'Weather in Boston for May 2025. Daily monitoring of air temperature and precipitation, daily and monthly temperature anomalies, temperature records in Boston. ... USA). Current location of the weather station: latitude 42.3606, longitude -71.0106, height above sea level 4 m. Explanations for calculating daily mean values.',
│   │   │   'score': 0.7845341,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Weather in Boston in May 2025 (Massachusetts) - Detailed Weather ...',
│   │   │   'url': 'https://world-weather.info/forecast/usa/boston/may-2025/',
│   │   │   'content': "World; United States; Massachusetts; Weather in Boston; Weather in Boston in May 2025. Boston Weather Forecast for May 2025 is based on long term prognosis and previous years' statistical data.",
│   │   │   'score': 0.6829338,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'National Weather Service',
│   │   │   'url': 'https://forecast.weather.gov/product.php?site=CRH&product=CF6&issuedby=BOS',
│   │   │   'content': 'noaa national weather service. 134 cxus51 kbox 050810 cf6bos preliminary local climatological data (ws form: f-6) station: boston ma month: may year: 2025 latitude',
│   │   │   'score': 0.6493346,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
The current weather in Boston is partly cloudy with a temperature of 17.2°C (63.0°F). The wind is blowing at 7.6 mph (12.2 km/h) from the east-southeast direction. The pressure is 1016 mb, and the humidity is 72%. There is no precipitation expected today.

========== Query processing completed ========== 



This output shows an AI agent processing a user query about "latest news OpenShift."

* **📍 Step 1: InferenceStep**: The agent *infers* the need to search for information and decides to use the `brave_search` tool with the query "latest news OpenShift."
* **📍 Step 2: ToolExecutionStep**: The agent *executes* the `brave_search` tool. The JSON block is the *output* from this tool, providing search results with titles, URLs, snippets, and relevance scores.
* **📍 Step 3: InferenceStep**: The agent *processes* the search results from the tool and *infers* how to construct a final answer.
* **🤖 Model Response**: This is the AI agent's *final generated response* to the user's query, summarizing the key information extracted from the search results.

Now lets try REact agents

### Creating our first custom tool

This code defines a custom tool called `get_location` for agent to use

* **`@client_tool`**: This decorator from `llama_stack_client.lib.agents.client_tool` marks the `get_location` function as a tool that the agent can discover and use to fulfill user requests. It essentially registers this function within the agent's toolkit.

* **Docstring Notes**: The text within the triple quotes (`"""Docstring goes here"""`) serves as crucial instructions for the AI agent. It explains:
    * **What the tool does**: "Provides the user's location upon request."
    * **How it works**: Mentions using the `geocoder` library and determining location via IP address.
    * **Input parameters**: Describes the `query` parameter.
    * **Output**: Specifies what the tool returns (location information as a string).
    * **Example**: Shows how the tool can be used and its expected output.

The agent reads this docstring to understand the tool's purpose, how to call it (with what arguments), and what kind of result to expect. This allows the agent to strategically use the `get_location` tool when a user asks for their location.

> **Note:**
> This is just one way to create a custom tool, we can also import tools from libraries or use MCP Servers (Again, more on this later...) 
> 

In [6]:
from llama_stack_client.lib.agents.client_tool import client_tool

@client_tool
def get_location(query: str = "location"):
    """
    Provides the user's location upon request.

    This function uses the geocoder library to determine the user's location
    based on their IP address.  It returns the city, state, and country.

    :param query:  The query from the user.  Defaults to "location".
    :type query: str
    :return:  Information about the user's current location.
    :rtype: str

    Example:
        >>> get_location("where am i")
        "Your current location is: Some City, Some State, Some Country"
    """
    import geocoder
    try:
        g = geocoder.ip('me')
        if g.ok:
            return f"Your current location is: {g.city}, {g.state}, {g.country}" # can be modified to return latitude and longitude if needed
        else:
            return "Unable to determine your location"
    except Exception as e:
        return f"Error getting location: {str(e)}"

## Prompt Chaining (Optional)

Prompt chaining allows an AI agent to remember information across multiple turns within the same session. In this example, both user questions share the same `session_id`.

Because they share a session, the agent will remember the answer to the first question ("Where am I?") when interpreting the second question ("search for any weather-related risks in my area..."). This enables the agent to understand "my area" refers to the location identified in the previous turn, leading to a more context-aware and helpful response.

### Defining a Multi-tool Agent

This code initializes an AI agent with specific instructions and tools. 
The `tools` parameter is a list containing elements the agent can use. Here, it includes `get_location` (a custom function defined earlier) and `"builtin::websearch"` (a pre-built web searching capability in Llama Stack). 


The instructions (often refered to as "Prompt" guide the agent on when to use each of these tools based on the user's query.




In [7]:

agent = Agent(
    client, 
    model=SELECTED_MODEL,
    # instructions="""You are a helpful assistant. 
    # When a user asks about their location, you MUST use the get_location tool. When searching for nearby places, you MUST use the websearch tool.
    # """ ,
    instructions="""You are a helpful assistant. 
    When a user asks about their location, you MUST use the get_location tool. When searching for nearby places, you MUST use the brave_search tool.
    """ ,
    tools=[get_location, "builtin::websearch"],
    #sampling_params=sampling_params
)

### inspecting the Agent's response structure with Prompt Chaining

Notice that we are **NOT** generating a new session ID for each question, this way, the convesation memory is shared throughtout the session. 


In [8]:
user_prompts = [
    "Where am I?",
    "search for any weather-related risks in my area that could disrupt network connectivity or system availability?",
]

# Generate a new Unique Identifier for the session, but *NOT* for each question.

new_uuid = uuid.uuid4()
session_id = agent.create_session("prompt-chaining-session-{new_uuid}")

for prompt in user_prompts:
    print("\n"+"="*50)
    print(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )

    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Where am I? blue

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: get_location, Arguments: {'query': 'location'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


'Your current location is: Columbus, Ohio, US'


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
<|python_tag|>{"city": "Columbus", "state": "Ohio", "country": "US"}

========== Query processing completed ========== 


Processing user query: search for any weather-related risks in my area that could disrupt network connectivity or system availability? blue

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'weather risks near Columbus, Ohio, US'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'weather risks near Columbus, Ohio, US',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in Columbus, Ohio, US',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Columbus', 'region': 'Ohio', 'country': 'United States of America', 'lat': 39.9611, 'lon': -82.9989, 'tz_id': 'America/New_York', 'localtime_epoch': 1746734955, 'localtime': '2025-05-08 16:09'}, 'current': {'last_updated_epoch': 1746734400, 'last_updated': '2025-05-08 16:00', 'temp_c': 19.4, 'temp_f': 66.9, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 11.6, 'wind_kph': 18.7, 'wind_degree': 14, 'wind_dir': 'NNE', 'pressure_mb': 1018.0, 'pressure_in': 30.05, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 66, 'cloud': 50, 'feelslike_c': 19.4, 'feelslike_f': 66.9, 'windchill_c': 17.5, 'windchill_f': 63.4, 'heatindex_c': 17.5, 'heatindex_f': 63.5, 'dewpoint_c': 14.3, 'dewpoint_f': 57.8, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 2.4, 'gust_mph': 13.8, 'gust_kph': 22.1}}",
│   │   │   'score': 0.835619,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Columbus, Ohio Climate Change Risks and Hazards: Precipitation, Heat',
│   │   │   'url': 'https://climatecheck.com/ohio/columbus',
│   │   │   'content': 'Columbus, OHTop Climate Change Risks: Precipitation, Heat People in Columbus, OH are especially likely to experience increased risks from precipitation and heat. Precipitation and heat risk in Columbus, OH is very high. Precipitation risk in Columbus, OH The annual precipitation in Columbus, OH is projected to increase from about 36.7" to about 39.4".Extreme precipitation in any form can pose significant risks. Heat risk in Columbus, OH Property owners can check a specific address for flood risk including FEMA flood zone, then take steps to reduce their vulnerability to flooding damage.Climate change is increasing inland and coastal flooding risk due to sea level rise and increasing chances of extreme precipitation. The risks presented on this page reflect modeled averages for Columbus, OH under one projected emissions scenario and can vary for individual properties.',
│   │   │   'score': 0.67243975,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Weather in Columbus in May 2025 (Ohio) - Detailed Weather Forecast for ...',
│   │   │   'url': 'https://world-weather.info/forecast/usa/columbus_1/may-2025/',
│   │   │   'content': "Weather in Columbus in May 2025 (Ohio) - Detailed Weather Forecast for a Month Weather World United States Weather in Columbus Weather in Columbus in May 2025 Columbus Weather Forecast for May 2025, is based on previous years' statistical data. +68°+57° +64°+54° +63°+55° +64°+57° +68°+57° +70°+59° +70°+59° +70°+59° +72°+63° +70°+59° +72°+63° +68°+57° +70°+59° +72°+59° +72°+59° +68°+59° +66°+57° +70°+59° +72°+61° +72°+63° +73°+63° +73°+63° +75°+64° +75°+64° +79°+68° +77°+68° +77°+68° +77°+68° +79°+68° +79°+70° +81°+70° Extended weather forecast in Columbus HourlyWeek10-Day14-Day30-DayYear Weather in large and nearby cities Weather in Washington, D.C.+48° Hilliard+45° Lima+37° Mansfield+41° Marion+41° New Philadelphia+43° Newark+41° Sunbury+45° Tiffin+39° Findlay+36° Chillicothe+48° Parkersburg+54° Amelia+52° Circleville+45° Dayton+48° Gallipolis+55° Dodds+48° Whisler+45° world's temperature today day day Copyright © 2025 «World-Weather.info» All rights reserved. Temperature units",
│   │   │   'score': 0.45871297,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Columbus, OH Weather Advisories - Warnings & Watches | AccuWeather',
│   │   │   'url': 'https://www.accuweather.com/en/us/columbus/43215/weather-warnings/350128',
│   │   │   'content': 'Columbus, OH Weather Advisories - Warnings & Watches | AccuWeather Columbus, OH Weather captions settings, opens captions settings dialog captions off, selected ### Hurrica


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Based on the search results, it appears that there are several weather-related risks in the Columbus, Ohio area that could disrupt network connectivity or system availability. The top risk is the potential for severe wind gusts, with a probability of 15% or greater within 25 miles of a point. Additionally, there is a high risk of precipitation, with an increase in annual precipitation projected to be around 39.4 inches, which could pose significant risks due to flooding. It is recommended to secure unsecured property to prevent wind damage and take steps to reduce vulnerability to flooding.

========== Query processing completed ========== 



## Lab Summary: Interacting with Agents and Tools

In this lab, you took your first steps in understanding and interacting with AI agents using the Llama Stack framework and observe the benefits of Llama Stack in providing a structured and intuitive way to build intelligent agents with access to a variety of tools, both built-in and custom, and the ability to manage conversational memory for more coherent interactions. 

You learned how to:

* **Define a basic AI agent:** You created an agent and provided it with instructions on how to behave.
* **Utilize built-in tools:** You explored how agents can leverage pre-existing tools within Llama Stack, specifically the `websearch` tool, to gather information from the web.
* **Observe agent behavior:** You became familiar with the concepts of "Turns" and "Steps" that an agent goes through when processing a query, including the crucial "Tool Call" step where the agent decides to use a tool.
* **Create custom tools:** You experienced how to extend an agent's capabilities by defining your own specialized tool (`get_location`) using Python and decorators.
* **Implement prompt chaining:** You explored how to maintain context across multiple user interactions within the same session, allowing the agent to remember previous information and use it to answer subsequent questions more effectively.

Through these exercises, you gained a foundational understanding of how agents operate within the Llama Stack ecosystem and how they can be equipped with tools to perform specific tasks. 